# Task
Following functions are given. Implement function calling that will be able to call the functions based on user question.

In [3]:
%pip install pytz pandas

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pytz
from datetime import datetime
import pandas as pd
import json
import math

def get_current_time(location):
    try:
        # Get the timezone for the city
        timezone = pytz.timezone(location)

        # Get the current time in the timezone
        now = datetime.now(timezone)
        current_time = now.strftime("%I:%M:%S %p")

        return current_time
    except:
        return "Sorry, I couldn't find the timezone for that location."
    

def get_stock_market_data(index):
    available_indices = [
        "S&P 500",
        "NASDAQ Composite",
        "Dow Jones Industrial Average",
        "Financial Times Stock Exchange 100 Index",
    ]

    if index not in available_indices:
        return "Invalid index. Please choose from 'S&P 500', 'NASDAQ Composite', 'Dow Jones Industrial Average', 'Financial Times Stock Exchange 100 Index'."

    # Read the CSV file
    data = pd.read_csv("stock_data.csv")

    # Filter data for the given index
    data_filtered = data[data["Index"] == index]

    # Remove 'Index' column
    data_filtered = data_filtered.drop(columns=["Index"])

    # Convert the DataFrame into a dictionary
    hist_dict = data_filtered.to_dict()

    for key, value_dict in hist_dict.items():
        hist_dict[key] = {k: v for k, v in value_dict.items()}

    return json.dumps(hist_dict)


def calculator(num1, num2, operator):
    if operator == "+":
        return str(num1 + num2)
    elif operator == "-":
        return str(num1 - num2)
    elif operator == "*":
        return str(num1 * num2)
    elif operator == "/":
        return str(num1 / num2)
    elif operator == "**":
        return str(num1**num2)
    elif operator == "sqrt":
        return str(math.sqrt(num1))
    else:
        return "Invalid operator"

In [2]:
print(get_current_time("America/New_York"))
print(get_stock_market_data("NASDAQ Composite"))
print(calculator(5, 5, "+"))

09:25:14 AM
{"Date": {"2": "2023-07-12", "3": "2023-07-13"}, "Open": {"2": 14000.65, "3": 14100.11}, "High": {"2": 14200.06, "3": 14250.0}, "Low": {"2": 13800.08, "3": 14000.67}, "Close": {"2": 14100.44, "3": 14050.81}, "Volume": {"2": 4000000, "3": 4200000}}
10


In [4]:
import os
import inspect
from openai import AzureOpenAI
from openai.types.chat import ChatCompletionMessageParam
import json
from dotenv import load_dotenv

load_dotenv()

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
  api_version="2024-02-01"
)

# helper method used to check if the correct arguments are provided to a function
def check_args(function, args):
    sig = inspect.signature(function)
    params = sig.parameters

    # Check if there are extra arguments
    for name in args:
        if name not in params:
            return False
    # Check if the required arguments are provided
    for name, param in params.items():
        if param.default is param.empty and name not in args:
            return False

    return True


In [ ]:
# Step 1: send the conversation and available functions to the model
messages : list[ChatCompletionMessageParam]  = []
messages.append({"role": "user", "content": "Która jest godzina w Nowym Jorku? Ile to 5 * pięć?"})

tools = [
   {
    "type": "function",
    "function": {
        "name": "get_current_time",
        "description": "Get the current time in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. Europe/London, Asia/Tokyo",
                }
            },
            "required": ["location"],
        },
    }
}, 
{
    "type": "function",
    "function": {
        "name": "get_stock_market_data",
        "description": "Retrieve historical stock market data for a given index",
        "parameters": {
            "type": "object",
            "properties": {
                "index": {
                    "type": "string",
                    "description": "The stock market index to retrieve data for, e.g., S&P 500, NASDAQ Composite, Dow Jones Industrial Average, Financial Times Stock Exchange 100 Index",
                    "enum": [
                        "S&P 500",
                        "NASDAQ Composite",
                        "Dow Jones Industrial Average",
                        "Financial Times Stock Exchange 100 Index"
                    ]
                }
            },
            "required": ["index"]
        }
    }
},
{
    "type": "function",
    "function": {
        "name": "calculator",
        "description": "Perform basic arithmetic operations",
        "parameters": {
            "type": "object",
            "properties": {
                "num1": {
                    "type": "number",
                    "description": "The first number in the calculation",
                },
                "num2": {
                    "type": "number",
                    "description": "The second number in the calculation",
                },
                "operator": {
                    "type": "string",
                    "description": "The arithmetic operator, e.g. +, -, *, /, **, sqrt",
                    "enum": ["+", "-", "*", "/", "**", "sqrt"]
                }
            },
            "required": ["num1", "operator"]
            
        }
    }
}
]


response = client.chat.completions.create(
    model=os.getenv("AZURE_OPENAI_COMPLETION_MODEL"),
    messages=messages,
    tools=tools,
    tool_choice="auto",  # auto is default, but we'll be explicit
)

response_message = response.choices[0].message
tool_calls = response_message.tool_calls

response_json = json.loads(response_message.model_dump_json())
print(json.dumps(response_json, indent=4))

{
    "content": null,
    "refusal": null,
    "role": "assistant",
    "audio": null,
    "function_call": null,
    "tool_calls": [
        {
            "id": "call_BZiJBlItoneh7i5Nwct8lwhh",
            "function": {
                "arguments": "{\"location\": \"America/New_York\"}",
                "name": "get_current_time"
            },
            "type": "function"
        },
        {
            "id": "call_7dVN3GsiVbcebXNTTo8wdO5m",
            "function": {
                "arguments": "{\"num1\": 5, \"num2\": 5, \"operator\": \"*\"}",
                "name": "calculator"
            },
            "type": "function"
        }
    ]
}


In [16]:

# Step 2: check if the model wanted to call a function
if tool_calls:
    # Step 3: call the function
    # Note: the JSON response may not always be valid; be sure to handle errors
    available_functions = {
        "get_current_time": get_current_time,
        "get_stock_market_data": get_stock_market_data,
        "calculator": calculator
    }  # only one function in this example, but you can have multiple
    messages.append(response_message)  # extend conversation with assistant's reply
    # Step 4: send the info for each function call and function response to the model
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_to_call = available_functions[function_name]
        function_args = json.loads(tool_call.function.arguments)
        if check_args(function_to_call, function_args) is False:
            print("Invalid number of arguments for function: " + function_name)
            break
        function_response = ""
        if function_name == "get_stock_market_data":
            function_response = function_to_call( index=function_args.get("index"))
        elif function_name == "calculator":
            function_response = function_to_call(
                num1=function_args.get("num1"),
                num2=function_args.get("num2"),
                operator=function_args.get("operator"),
            )
        elif function_name == "get_current_time":
            function_response = function_to_call(location=function_args.get("location"))
        else:            
            print("Function not recognized: " + function_name)
            break

        messages.append(
            {
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response
    
    
    second_response = client.chat.completions.create(
        model=os.getenv("AZURE_OPENAI_COMPLETION_MODEL"),
        messages=messages,
    )  # get a new response from the model where it can see the function response
    
    second_response_json = json.loads(second_response.model_dump_json())
    #print(json.dumps(second_response_json, indent=4))
    print(second_response.choices[0].message.content)

W Nowym Jorku jest godzina 10:08:39 AM. A 5 * pięć to 25.


In [17]:
def custom_serializer(obj):
    if hasattr(obj, "model_dump"):
        return obj.model_dump()
    elif hasattr(obj, "__dict__"):
        return obj.__dict__
    else:
        return str(obj)

print(json.dumps(messages, indent=4, default=custom_serializer))


[
    {
        "role": "user",
        "content": "Kt\u00f3a jest godzina w Nowym Jorku? Ile to 5 * pi\u0119\u0107?"
    },
    {
        "content": null,
        "refusal": null,
        "role": "assistant",
        "audio": null,
        "function_call": null,
        "tool_calls": [
            {
                "id": "call_BZiJBlItoneh7i5Nwct8lwhh",
                "function": {
                    "arguments": "{\"location\": \"America/New_York\"}",
                    "name": "get_current_time"
                },
                "type": "function"
            },
            {
                "id": "call_7dVN3GsiVbcebXNTTo8wdO5m",
                "function": {
                    "arguments": "{\"num1\": 5, \"num2\": 5, \"operator\": \"*\"}",
                    "name": "calculator"
                },
                "type": "function"
            }
        ]
    },
    {
        "tool_call_id": "call_BZiJBlItoneh7i5Nwct8lwhh",
        "role": "tool",
        "name": "get_current